<a href="https://colab.research.google.com/github/SophieShin/MachineLearning/blob/main/Option_Pricing(%2Bprc_lmt).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 반드시 사본 저장 후 사본파일에서 실행해주세요.(파일/Drive에 사본저장)
#### 아래 '재생'버튼을 누르면서 진행하면 됩니다


In [1]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np

#### [CR20]화면에서 검증하고자 하는 상품의 이론가격 파라미터를 저장합니다
#### 결제월 : 전체
#### 선물옵션구분: 콜+풋
#### 이론가용도: 기준가용
#### 화면에서 프린트 버튼을 누른 뒤 엑셀로 저장하면 됩니다.
#### 엑셀로 저장할 때  'CSV' 형식으로 저장합니다.
#### 엑셀로 저장할 때 파라미터를 제외한 문구들(맨 위에 화면구성, 맨 아래에 주석 등)을 지우고 작업하는게 편합니다

In [2]:
# 파일 업로드
# 예제파일 2개(이론가파람_SK하이닉스, 상하한가) 파일을 https://github.com/SophieShin/MachineLearning/tree/main 에서 다운 받을 수 있습니다.

from google.colab import files

uploaded = files.upload()

Saving LG디스플레이.csv to LG디스플레이.csv


#### 아래 uploaded['이론가파람_SK하이닉스.csv'] 부분에서 한글 부분을 자신이 저장한 파일 이름으로 변경합니다

In [3]:
import io
df = pd.read_csv(io.BytesIO(uploaded['LG디스플레이.csv']), encoding='utf-8', thousands = ',')
df

,종목명,선물옵션구분,최종거래일,행사가격,기초자산\r\r\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
0,"C 202308 12,500( 10)",콜,2023-08-10,12500.0,15080.0,28,3.75,0,0.293,2619.34
1,"C 202308 13,000( 10)",콜,2023-08-10,13000.0,15080.0,28,3.75,0,0.293,2130.57
2,"C 202308 13,500( 10)",콜,2023-08-10,13500.0,15080.0,28,3.75,0,0.293,1661.13
3,"C 202308 14,000( 10)",콜,2023-08-10,14000.0,15080.0,28,3.75,0,0.293,1228.44
4,"C 202308 14,500( 10)",콜,2023-08-10,14500.0,15080.0,28,3.75,0,0.293,852.00
...,...,...,...,...,...,...,...,...,...,...
161,"P 202406 17,000( 10)",풋,2024-06-13,17000.0,15080.0,336,3.75,0,0.299,2539.34
162,"P 202406 18,000( 10)",풋,2024-06-13,18000.0,15080.0,336,3.75,0,0.299,3233.27
163,"P 202406 19,000( 10)",풋,2024-06-13,19000.0,15080.0,336,3.75,0,0.299,3973.57
164,"P 202406 20,000( 10)",풋,2024-06-13,20000.0,15080.0,336,3.75,0,0.299,4756.61


In [4]:
# null 값 제거
df= df.dropna(subset=['종목명'])

In [5]:
# 금리 단위 맞추기
df['금리(%)']= df['금리(%)']/100

In [6]:
# 풋옵션 이론가 산출 파라미터만 남기기
df2 = df[df['선물옵션구분'] == '풋']

In [8]:
# 콜옵션 이론가 산출 파라미터만 남기기
df = df[df['선물옵션구분'] == '콜']

#### 1. 콜옵션 이론가 구하기

In [9]:
# null값 제거
df = df.dropna(axis = 1)
df

,종목명,선물옵션구분,최종거래일,행사가격,기초자산\r\r\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
0,"C 202308 12,500( 10)",콜,2023-08-10,12500.0,15080.0,28,0.0375,0,0.293,2619.34
1,"C 202308 13,000( 10)",콜,2023-08-10,13000.0,15080.0,28,0.0375,0,0.293,2130.57
2,"C 202308 13,500( 10)",콜,2023-08-10,13500.0,15080.0,28,0.0375,0,0.293,1661.13
3,"C 202308 14,000( 10)",콜,2023-08-10,14000.0,15080.0,28,0.0375,0,0.293,1228.44
4,"C 202308 14,500( 10)",콜,2023-08-10,14500.0,15080.0,28,0.0375,0,0.293,852.00
...,...,...,...,...,...,...,...,...,...,...
78,"C 202406 17,000( 10)",콜,2024-06-13,17000.0,15080.0,336,0.0375,0,0.293,1160.10
79,"C 202406 18,000( 10)",콜,2024-06-13,18000.0,15080.0,336,0.0375,0,0.293,890.38
80,"C 202406 19,000( 10)",콜,2024-06-13,19000.0,15080.0,336,0.0375,0,0.293,669.44
81,"C 202406 20,000( 10)",콜,2024-06-13,20000.0,15080.0,336,0.0375,0,0.293,486.64


In [10]:
# 이론가격을 구하기 위해 data 자료타입 변경

S = list(map(int,df['기초자산\r\r\n기준가격'].tolist()))
K = list(map(int,df['행사가격'].tolist()))
r = list(map (float,df['금리(%)'].tolist()))
div = list(map(int,df['배당  현재가치'].tolist()))
sigma = list(map(float,df['변동성'].tolist()))
t = list(map(int,df['잔존일수'].tolist()))
theo_prc_sys = list(map(float,df['이론가격'].tolist()))
name = df['종목명'].tolist()

In [12]:
# 상하한가 계산을 위해 기초자산 가격을 가격제한폭으로 설정하고 자료타입 변경
upper_lmt =  df['기초자산\r\r\n기준가격']*1.3
lower_lmt =  df['기초자산\r\r\n기준가격']*0.7
upper_lmt = list(map(int,upper_lmt.tolist()))
lower_lmt = list(map(int,lower_lmt.tolist()))

In [13]:
# call 옵션 이론가격 구하는 코드

def call(s,k,R,T,Div, Sigma):
  u = np.exp(Sigma*np.sqrt(T/365/49))
  d = np.exp(-Sigma*np.sqrt(T/365/49))
  p = (np.exp(R *T/365/49) - d)/(u-d)
  q = 1-p
  y = []

  for i in range(50):
    C_i = max((s-Div) * u**i * d**(49-i) -k , 0)
    x_i = np.math.factorial(49)/(np.math.factorial(49-i)*np.math.factorial(i))
    p_i = p**i
    q_i = q**(49-i)
    y.append(C_i*x_i* p_i*q_i)

  return sum(y)*np.exp(-R*T/365)

In [14]:
# 모든 행사가격, 모든 월물에 대해 이론가격 구하는 코드
call_price_temp = []

for i in range(len(name)):
  s = S[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]

  price =  call(s,k,R,T,Div, Sigma)
  call_price_temp.append(price)


In [15]:
# 상한가 구하기
upper_lmt_temp = []

for i in range(len(name)):
  s = upper_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]*2

  upper_price =  call(s,k,R,T,Div, Sigma)
  upper_lmt_temp.append(upper_price)


In [16]:
# 하한가 구하기
lower_lmt_temp =[]

for i in range(len(name)):
  s = lower_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = 0.03

  lower_price = call(s,k,R,T,Div, Sigma)
  lower_lmt_temp.append(lower_price)

In [17]:
# 구한 이론가와 시스템에서 산출한 이론가 비교, 상하한가 표출

call_price = pd.Series(call_price_temp)
upper_limit = pd.Series(upper_lmt_temp)
lower_limit = pd.Series(lower_lmt_temp)
df['theo_price_검증용'] = call_price
df['시스템 검증 차이'] = df['이론가격'] - df['theo_price_검증용']
df['상한가'] = upper_limit
df['하한가'] = lower_limit

In [18]:
df

,종목명,선물옵션구분,최종거래일,행사가격,기초자산\r\r\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가
0,"C 202308 12,500( 10)",콜,2023-08-10,12500.0,15080.0,28,0.0375,0,0.293,2619.34,2619.336475,0.003525,7141.641770,0.0
1,"C 202308 13,000( 10)",콜,2023-08-10,13000.0,15080.0,28,0.0375,0,0.293,2130.57,2130.573982,-0.003982,6645.494448,0.0
2,"C 202308 13,500( 10)",콜,2023-08-10,13500.0,15080.0,28,0.0375,0,0.293,1661.13,1661.131249,-0.001249,6151.688578,0.0
3,"C 202308 14,000( 10)",콜,2023-08-10,14000.0,15080.0,28,0.0375,0,0.293,1228.44,1228.442001,-0.002001,5661.423927,0.0
4,"C 202308 14,500( 10)",콜,2023-08-10,14500.0,15080.0,28,0.0375,0,0.293,852.00,851.999637,0.000363,5175.702553,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,"C 202406 17,000( 10)",콜,2024-06-13,17000.0,15080.0,336,0.0375,0,0.293,1160.10,1160.101222,-0.001222,5784.537316,0.0
79,"C 202406 18,000( 10)",콜,2024-06-13,18000.0,15080.0,336,0.0375,0,0.293,890.38,890.384571,-0.004571,5276.655739,0.0
80,"C 202406 19,000( 10)",콜,2024-06-13,19000.0,15080.0,336,0.0375,0,0.293,669.44,669.443757,-0.003757,4868.088925,0.0
81,"C 202406 20,000( 10)",콜,2024-06-13,20000.0,15080.0,336,0.0375,0,0.293,486.64,486.639578,0.000422,4469.462816,0.0


In [19]:
print('시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목')
df[df['시스템 검증 차이'].abs() > 1]

## 아래에 칼럼명만 나오면 1원 이상 차이나는 종목이 없다는 뜻입니다.

시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목


,종목명,선물옵션구분,최종거래일,행사가격,기초자산\r\r\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가


#### 2. 풋옵션 이론가 구하기

In [20]:
# null값 제거
df2 = df2.dropna(axis = 1)
df2

,종목명,선물옵션구분,최종거래일,행사가격,기초자산\r\r\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
83,"P 202308 12,500( 10)",풋,2023-08-10,12500.0,15080.0,28,0.0375,0,0.299,4.02
84,"P 202308 13,000( 10)",풋,2023-08-10,13000.0,15080.0,28,0.0375,0,0.299,15.21
85,"P 202308 13,500( 10)",풋,2023-08-10,13500.0,15080.0,28,0.0375,0,0.299,45.59
86,"P 202308 14,000( 10)",풋,2023-08-10,14000.0,15080.0,28,0.0375,0,0.299,114.70
87,"P 202308 14,500( 10)",풋,2023-08-10,14500.0,15080.0,28,0.0375,0,0.299,238.55
...,...,...,...,...,...,...,...,...,...,...
161,"P 202406 17,000( 10)",풋,2024-06-13,17000.0,15080.0,336,0.0375,0,0.299,2539.34
162,"P 202406 18,000( 10)",풋,2024-06-13,18000.0,15080.0,336,0.0375,0,0.299,3233.27
163,"P 202406 19,000( 10)",풋,2024-06-13,19000.0,15080.0,336,0.0375,0,0.299,3973.57
164,"P 202406 20,000( 10)",풋,2024-06-13,20000.0,15080.0,336,0.0375,0,0.299,4756.61


In [21]:
#인덱스 초기화
df2.reset_index(inplace = True)

In [22]:
df2

,index,종목명,선물옵션구분,최종거래일,행사가격,기초자산\r\r\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격
0,83,"P 202308 12,500( 10)",풋,2023-08-10,12500.0,15080.0,28,0.0375,0,0.299,4.02
1,84,"P 202308 13,000( 10)",풋,2023-08-10,13000.0,15080.0,28,0.0375,0,0.299,15.21
2,85,"P 202308 13,500( 10)",풋,2023-08-10,13500.0,15080.0,28,0.0375,0,0.299,45.59
3,86,"P 202308 14,000( 10)",풋,2023-08-10,14000.0,15080.0,28,0.0375,0,0.299,114.70
4,87,"P 202308 14,500( 10)",풋,2023-08-10,14500.0,15080.0,28,0.0375,0,0.299,238.55
...,...,...,...,...,...,...,...,...,...,...,...
78,161,"P 202406 17,000( 10)",풋,2024-06-13,17000.0,15080.0,336,0.0375,0,0.299,2539.34
79,162,"P 202406 18,000( 10)",풋,2024-06-13,18000.0,15080.0,336,0.0375,0,0.299,3233.27
80,163,"P 202406 19,000( 10)",풋,2024-06-13,19000.0,15080.0,336,0.0375,0,0.299,3973.57
81,164,"P 202406 20,000( 10)",풋,2024-06-13,20000.0,15080.0,336,0.0375,0,0.299,4756.61


In [24]:
# 이론가격을 구하기 위해 data 자료구조 변경

S = list(map(int,df2['기초자산\r\r\n기준가격'].tolist()))
K = list(map(int,df2['행사가격'].tolist()))
r = list(map (float,df2['금리(%)'].tolist()))
div = list(map(int,df2['배당  현재가치'].tolist()))
sigma = list(map(float,df2['변동성'].tolist()))
t = list(map(int,df2['잔존일수'].tolist()))
theo_prc_sys = list(map(float,df2['이론가격'].tolist()))
name = df2['종목명'].tolist()

In [25]:
# 상하한가 계산을 위해 기초자산 가격을 가격제한폭으로 설정하고 자료타입 변경
upper_lmt =  df['기초자산\r\r\n기준가격']*0.7
lower_lmt =  df['기초자산\r\r\n기준가격']*1.3
upper_lmt = list(map(int,upper_lmt.tolist()))
lower_lmt = list(map(int,lower_lmt.tolist()))

In [26]:
# put 옵션 이론가격 구하는 코드

def put(s,k,R,T,Div, Sigma):
  u = np.exp(Sigma*np.sqrt(T/365/49))
  d = np.exp(-Sigma*np.sqrt(T/365/49))
  p = (np.exp(R *T/365/49) - d)/(u-d)
  q = 1-p
  y = []

  for i in range(50):
    P_i = max(k -(s-Div)* u**i * d**(49-i)  , 0)
    x_i = np.math.factorial(49)/(np.math.factorial(49-i)*np.math.factorial(i))
    p_i = p**i
    q_i = q**(49-i)
    y.append(P_i*x_i* p_i*q_i)

  return sum(y)*np.exp(-R*T/365)

In [27]:
# 모든 행사가격, 모든 월물에 대해 이론가격 구하는 코드

put_price_temp = []

for i in range(len(name)):
  s = S[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]

  price =  put(s,k,R,T,Div,Sigma)
  put_price_temp.append(price)


In [28]:
# 상한가 구하기
upper_lmt_temp = []

for i in range(len(name)):
  s = upper_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = sigma[i]*2

  upper_price =  put(s,k,R,T,Div, Sigma)
  upper_lmt_temp.append(upper_price)

In [29]:
# 하한가 구하기
lower_lmt_temp =[]

for i in range(len(name)):
  s = lower_lmt[i]
  k = K[i]
  R = r[i]
  T = t[i]
  Div = div[i]
  Sigma = 0.03

  lower_price = put(s,k,R,T,Div, Sigma)
  lower_lmt_temp.append(lower_price)

In [30]:
# 구한 이론가와 시스템에서 산출한 이론가 비교, 상하한가 표출

call_price = pd.Series(put_price_temp)
upper_limit = pd.Series(upper_lmt_temp)
lower_limit = pd.Series(lower_lmt_temp)
df2['theo_price_검증용'] = call_price
df2['시스템 검증 차이'] = df2['이론가격'] - df2['theo_price_검증용']
df2['상한가'] = upper_limit
df2['하한가'] = lower_limit

In [31]:
df2

,index,종목명,선물옵션구분,최종거래일,행사가격,기초자산\r\r\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가
0,83,"P 202308 12,500( 10)",풋,2023-08-10,12500.0,15080.0,28,0.0375,0,0.299,4.02,4.017874,0.002126,2063.040105,0.000000e+00
1,84,"P 202308 13,000( 10)",풋,2023-08-10,13000.0,15080.0,28,0.0375,0,0.299,15.21,15.209446,0.000554,2505.105544,0.000000e+00
2,85,"P 202308 13,500( 10)",풋,2023-08-10,13500.0,15080.0,28,0.0375,0,0.299,45.59,45.587532,0.002468,2967.332943,0.000000e+00
3,86,"P 202308 14,000( 10)",풋,2023-08-10,14000.0,15080.0,28,0.0375,0,0.299,114.70,114.697109,0.002891,3441.567064,0.000000e+00
4,87,"P 202308 14,500( 10)",풋,2023-08-10,14500.0,15080.0,28,0.0375,0,0.299,238.55,238.552097,-0.002097,3924.209956,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,161,"P 202406 17,000( 10)",풋,2024-06-13,17000.0,15080.0,336,0.0375,0,0.299,2539.34,2539.339631,0.000369,6807.772398,8.434957e-09
79,162,"P 202406 18,000( 10)",풋,2024-06-13,18000.0,15080.0,336,0.0375,0,0.299,3233.27,3233.266327,0.003673,7639.449810,1.476922e-03
80,163,"P 202406 19,000( 10)",풋,2024-06-13,19000.0,15080.0,336,0.0375,0,0.299,3973.57,3973.573649,-0.003649,8484.872958,2.044665e+00
81,164,"P 202406 20,000( 10)",풋,2024-06-13,20000.0,15080.0,336,0.0375,0,0.299,4756.61,4756.608956,0.001044,9368.616983,1.073225e+02


In [32]:
print('시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목')
df2[df2['시스템 검증 차이'].abs() > 1]

## 아래에 칼럼명만 나오면 1원 이상 차이나는 종목이 없다는 뜻입니다.

시스템 이론가격과 검증이론가격의 차이가 1원 이상 나는 종목


,index,종목명,선물옵션구분,최종거래일,행사가격,기초자산\r\r\n기준가격,잔존일수,금리(%),배당 현재가치,변동성,이론가격,theo_price_검증용,시스템 검증 차이,상한가,하한가
